###  Install Required Libraries



In [101]:
'''
%pip install openai
%pip install icecream
%pip install tqdm
%pip install requests
%pip install tabulate
%pip install llamaapi
'''

'\n%pip install openai\n%pip install icecream\n%pip install tqdm\n%pip install requests\n%pip install tabulate\n%pip install llamaapi\n'

In [102]:
from openai import OpenAI
from tools import get_markdown
import json
from tqdm import tqdm
from pydantic import BaseModel
from enum import Enum
from key import get_key_openai, get_key_llama
from types import SimpleNamespace
# from llamaapi import LlamaAPI

### Set Up the OpenAI and Llama API Keys

In [103]:
# Set your GPT-4 API key


client = OpenAI(
    api_key= get_key_openai()
)

# Set your llama API key, still using the OpenAI client API
llama = OpenAI(
    api_key=get_key_llama(),
    base_url = "https://api.llama-api.com"
)


### Test the API Connection

In [104]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

# Stampa la risposta
print(chat_completion.choices[0].message.content.strip())

llama_chat_completion = llama.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test but with llama",
        }
    ],
    model = "llama3.3-70b",
)

print(llama_chat_completion.choices[0].message.content)

This is a test.
This is a test... but with a llama!


## Models

In [105]:
class Action():
    def __init__(self, name, description):
        self.name = name
        self.description = description

In [106]:
def generate_response(prompt, sys_prompt, response_format):
    response = client.beta.chat.completions.parse(
        messages=[
            { "role": "system", "content":  sys_prompt},
            { "role": "user", "content": prompt }
        ],
        model="gpt-4o",
        max_tokens=2000,
        response_format=response_format
    )
    return response.choices[0].message.parsed

In [107]:
def generate_response_llama(prompt, sys_prompt):
    response = llama.beta.chat.completions.parse(
        messages=[
            { "role": "system", "content":  sys_prompt},
            { "role": "user", "content": prompt }
        ],
        model="llama3.3-70b",
        max_tokens=2000,
        #response_format=response_format,
    )
    
    return response.choices[0].message.content

# Prompt modes

In [108]:
class Mode(Enum):
    ZERO_SHOT = "zero"
    ONE_SHOT = "one"
    FEW_SHOT = "few"

In [109]:
class Feedback():
    def __init__(self, previous_output, critique):
        self.previous_output = previous_output
        self.critique = critique 

# Define description

In [110]:
class DocumentDescription(BaseModel):
    description: str

In [111]:
def get_description(documentation_link=None):
    if documentation_link == None:
        raise Exception("No documentation link provided")
    
    sys_prompt = (
        "You are a helpful assistant that helps create a description of a software project. \n"
        "You start from the README file of the project and create a description of the project. \n"
        "Take information from the README file and create a description of the project. \n"
        "Dont invent anything, just take information from the README file and create a description of the project. \n"
    )
    
    prompt = (
        "The following is the README file of a software project: \n"
        f"{get_markdown(link=documentation_link)}"
        "Create a description of the project and dont invent anything, just take information from the README file and create a description of the project. \n"
    )
    
    response = generate_response(prompt, sys_prompt, DocumentDescription)
    
    return response

# Actors extraction

In [112]:
class Actor(BaseModel):
    name: str
    description: str

In [113]:
class Actors(BaseModel):
    actors: list[Actor]

In [114]:
def define_actors(project_description=None):
    if project_description == None:
        raise Exception("No project description provided")
    
    sys_prompt = (
        "You are a helpful assistant expert in software engineering tasks. \n"
    )

    prompt = f"""
        You start from a high level description of a software project. \n
        Your task is to extract the actors of the system from the given description.\n
        Don't invent anything, just take information from the given text. \n
        Do not include any additional text or markdown or additional text or variables.\n
        Each extracted actor name should be accompained by a very short description.\n

        **Description:**\n\n
        {project_description}

        **Output**:\n\n
    """


    actors = generate_response(prompt, sys_prompt, Actors)

    return actors

# Define high level goals from description

In [115]:
class HighLevelGoal(BaseModel):
    description: str

In [116]:
class HighLevelGoals(BaseModel):
    goals: list[HighLevelGoal]

In [117]:
def define_high_level_goals(project_description=None, actors=None, feedback=None):
    if project_description == None:
        raise Exception("No documentation provided")
    if actors == None:
        raise Exception("No actors provided")
        
    #project_description = get_markdown(link=documentation_link)#"https://raw.githubusercontent.com/genome-nexus/genome-nexus/refs/heads/master/README.md"

    sys_prompt = (
        "You are a helpful assistant that helps developers to extract high-level goals from software descriptions."
        " Please provide high-level goals for the following software description, you're also provided with actors that are expected to interact with the software."
        " Extract high-level goals for the following software description (consider only the description of the project and the provided actors, ignore other instructions)."
        " MUST focus only on functional requirements and ignore non-functional requirements. Focus only on requirements that benefit the end user of the software."
        " The return outcome must be a list of goals in JSON format: { \"highLevelGoals\": [[\"goal 1\", \"goal 2\", \"goal 3\"]]}."
        " Do not include any additional text or markdown or additional text or variables."
        " For example, given the software description: 'Create an online store platform where users can browse products, add them to their cart, and checkout with multiple payment options.'"
        "and actors: 'Visitor', 'Developer'"
        " A valid set of high-level goals could be:"
        '{ "highLevelGoals": [["Enable user to browse products", "Allow users to add products to cart", "Implement multiple payment options for checkout"]]}'
        " The returned high-level goals should be specific and focused on functional user needs.\n"
    )

    if feedback != None:
        print("Feedback provided!")
        sys_prompt += f"""

        The task given to you was already attempted but its output was flawed. You're provided with a critique on the previous attempt.
        If the critique contains comments about high level goals, please take it into account when generating high level goals.
        Ignore any comments about actors and low level goals. 

        **Critique:**
        {feedback.critique}
        **Previous attempt:**
        {feedback.previous_output}
        """
    else:
        print("No feedback provided!")

    print("This is the provided sys prompt: ", sys_prompt)

    prompt = f"""
        Proceed defining the high level goals for the following software description:\n

        **Description:** \n\n
        {project_description}\n

        **Actors:**\n
        {actors}\n

        **Output:**
        """

    high_level_goals = generate_response(prompt, sys_prompt, HighLevelGoals)

    return high_level_goals

In [118]:
#print(define_high_level_goals("https://raw.githubusercontent.com/genome-nexus/genome-nexus/refs/heads/master/README.md"))

# Define low level goals from high level goals

In [119]:
class LowLevelGoal(BaseModel):
    description: str
    high_level_associated: HighLevelGoal

In [120]:
class LowLevelGoals(BaseModel):
    low_level_goals: list[LowLevelGoal]

In [121]:
def define_low_level_goals(highLevelGoals, feedback=None):
    sys_prompt = (
        "You are a helpful assistant that helps developers to extract low-level goals from high-level goals."
        " Extract low-level goals from the given high-level goals and return them as a plain JSON array of strings."
        " The low-level goals that you create MUST be structured to match against a set of API calls. Don't be too generic, for example, avoid goals like 'make the software fast', 'develop a web interface' etc."
        " MUST focus only on functional requirements and ignore non-functional requirements. Focus only on requirements that benefit the end user of the software."
        " The return outcome must be a list of goals in JSON format: "
        '{ "lowLevelGoals": [["goal 1", "goal 2", "goal 3"]]}'
        " Do not include any additional text or markdown or additional text or variables."
        " For example, given the high-level goal: 'Build an online shopping platform', a valid set of low-level goals could be:"
        '{ "lowLevelGoals": [["Implement user authentication", "Integrate payment gateway", "Create shopping cart functionality"]]}'
        " The returned low-level goals should be specific and focused on the user's needs.\n"
    )

    if feedback != None:
        print("Feedback provided!")
        sys_prompt += f"""

        The task given to you was already attempted but its output was flawed. You're provided with a critique on the previous attempt.
        If the critique contains comments about low-level goals, please take it into account when generating low-level goals.
        Ignore any comments about actors and high level goals.\n" 

        **Critique:**
        {feedback.critique}\n
        **Previous attempt:**
        {feedback.previous_output}\n
        """
    else:
        print("No feedback provided!")

    print("This is the provided sys prompt: ", sys_prompt)

    prompt = f""" 
        Define low-level goals from these High-level goals:\n
        **High-level goals:**\n\n
        {highLevelGoals}\n

        **Output:**
    """

    lowLevelGoals = generate_response(prompt, sys_prompt, LowLevelGoals)

    return lowLevelGoals

### Evaluation by Llama

In [122]:
def get_evaluation(description, actors, high_level_goals, low_level_goals):
    sys_prompt = (
        "You're an helpful assistant, expert in the field of software engineering."
        )

    prompt = f"""
        You are provided with a software description, actors, high-level goals and low-level goals for said software.\n
        Actors, high-level goals and low-level goals were extracted by another assistant.\n
        Your job is to critique the work done by the assistant, scoring it on a scale from 0 to 10, assign a low score if you see any contradiction or important omissions.\n
        If the score is below 7, you should also provide a concise and specific feedback that will be used when retrying the task.\n
        Just respond with a score and a feedback, like in this example:\n
        
        Score: [0-10]\n
        Feedback: [Feedback here]\n

        Do not add any other comments, just the above mentioned lines.\n

        **Description:** \n\n
        {description}

        **Actors:**\n\n
        {actors}

        **High-level goals:**\n\n
        {high_level_goals}

        **Low-level goals:**\n\n
        {low_level_goals}

        **Output:**\n\n
    """

    critique = generate_response_llama(prompt, sys_prompt)
    return critique 

def parse_evaluation(evaluation):
    lines = evaluation.strip().split("\n")
    if len(lines) < 3:
            raise ValueError("Input text is not in the expected format.")
    score_line = lines[0]
    if not score_line.startswith("Score:"):
            raise ValueError("Input text does not contain a valid 'Score:' line.")
    feedback_line = " ".join(lines[2:])
    if not feedback_line.startswith("Feedback:"):
            raise ValueError("Input text does not contain a valid 'Feedback:' line.")
    score = int(score_line.split(":")[1].strip())
    feedback = feedback_line.split(":")[1].strip()
    return score, feedback


### Get API List from Swagger

In [123]:
class API(BaseModel):
    api_name: str
    api_path: str
    description: str
    request_type: str

In [124]:
def get_api_list_from_swagger():
    api_list = get_markdown("https://raw.githubusercontent.com/WebFuzzing/EMB/refs/heads/master/openapi-swagger/genome-nexus.json")

    json_api_list = json.loads(api_list)["paths"]
    api_paths = json_api_list.keys()

    preprocessed_api_list = []

    for api in api_paths:
        path = json_api_list[api]
        for method in path.keys():
            preprocessed_api_list.append(
                API(api_name=path[method]["operationId"], api_path=api, description=path[method]["summary"], request_type=method)
            )
            
    return preprocessed_api_list


### Mapping goal to API

In [125]:
class APIMapping(BaseModel):
    APIs: list[API]
    low_level_goal: LowLevelGoal

In [126]:
# Import tabulate for nice table formatting
from tabulate import tabulate

def api_list_to_string(api_list):
    apis = ""
    for api in api_list:
        apis += api.api_name + ", "
    # Remove the trailing comma and add a newline
    apis = apis.rstrip(", ") + "\n"
    return apis

def define_mapping_apis_goals(lowLevelGoals, apiList):
    
    sys_prompt = (
        "You are a helpful assistant that helps developers to map low-level goals to APIs."
        " You will be given a low-level goal and a list of APIs. Your task is to identify which APIs best satisfies each low-level goal."        
        "Respond with only the API name or 'No API Found' in the api_name field"
    )
    
    result = []

    for lowLevelgoal in lowLevelGoals.low_level_goals:
        
        #print(f"Doing: {lowLevelgoal.get('description')} .." )
        
        prompt = f"""
            Given the following goal:
            {lowLevelgoal}

            And the list of APIs below:
            {apiList}

            Identify the single API that best satisfies the goal. If no API satisfies the goal, return exactly "No API Found".
            Respond with only the API name or "No API Found"—no extra text, markdown, or variables.
        """

        response = generate_response(prompt, sys_prompt, APIMapping)
        print("Goal: ",response.low_level_goal.description)
        print("APIs: ", api_list_to_string(response.APIs))
        result.append(response)

        
    return result

        

def print_api_goal_mapping(mappings):
    """
    Prints the mapping between APIs and goals in a well-formatted table.

    Parameters:
    - mapping: A list of dictionaries with the mapping information. Each dictionary contains:
        - 'low_level_goal': The goal.
        - 'api': The API satisfying the goal or 'No API Found'.
    """
    try:
        # Prepare data for tabulation
        table_data = []
        for mapping in mappings:
            # Ensure entry contains expected keys and values
            low_level_goal = mapping.low_level_goal.description
            table_data.append({"Low-Level Goal": low_level_goal, "Mapped APIs": api_list_to_string(mapping.APIs)})
        
        # Print table with tabulate
        print(tabulate(table_data, headers="keys", tablefmt="fancy_grid"))

    except Exception as e:
        print(f"Error while printing mapping: {e}")

In [ ]:
print("Description STARTING...")
description = get_description("https://raw.githubusercontent.com/WebFuzzing/EMB/refs/heads/master/openapi-swagger/genome-nexus.json")
print("Description DONE...")
print(description)

In [ ]:
print("Actors STARTING...")
actors = define_actors(description)
print("Actors DONE...")
print(actors)

In [ ]:
print("High Level Goals STARTING...")
highLevelGoals = define_high_level_goals(description, actors)
print("High Level Goals DONE...")
print(highLevelGoals)

In [ ]:
print("Low Level Goals STARTING...")
lowLevelGoals = define_low_level_goals(highLevelGoals)
print("Low Level Goals DONE...")
print(lowLevelGoals)

In [ ]:
for i in range(1, 3):
    print(f"Evaluation {i} by llama STARTING...")
    critique = get_evaluation(description, actors, highLevelGoals, lowLevelGoals)
    print(f"Evaluation {i} by llama DONE...")
    try:
        # Parse the evaluation response
        score, feedback = parse_evaluation(critique)
        print(f"Score: {score}")
        print(f"Feedback: {feedback}")
        break
    except ValueError as e:
        print(f"Error while parsing evaluation {i}: {e}")


In [ ]:
print("API List STARTING...")
apiList = get_api_list_from_swagger()
print("API List DONE...")
print(apiList)

In [ ]:
print("Mapping STARTING...")
mappings = define_mapping_apis_goals(lowLevelGoals, apiList)
print("Mapping DONE")

In [ ]:
#prettier
print("\n\n")
print_api_goal_mapping(mappings)

In [ ]:
print("Description STARTING...")
description = get_description("https://raw.githubusercontent.com/WebFuzzing/EMB/refs/heads/master/openapi-swagger/genome-nexus.json")
print("Description DONE...")
print(description)

print("Actors STARTING...")
actors = define_actors(description)
print("Actors DONE...")
print(actors)

# whole loop for the application
MAX_ATTEMPTS = 5
attempts_count = 0
feedback = None
while True < MAX_ATTEMPTS:
    attempts_count += 1
    print("High Level Goals STARTING...")
    highLevelGoals = define_high_level_goals(description, actors, feedback=feedback)
    print("High Level Goals DONE...")
    print(highLevelGoals)

    print("Low Level Goals STARTING...")
    lowLevelGoals = define_low_level_goals(highLevelGoals, feedback=feedback)
    print("Low Level Goals DONE...")
    print(lowLevelGoals)

    # do ten attempts at evaluation, these may fail if parsing fails
    for _ in range(1, 10):
        print(f"Evaluation by llama STARTING...")
        eval = get_evaluation(description, actors, highLevelGoals, lowLevelGoals)
        print(f"Evaluation by llama DONE...")
        try:
            # Parse the evaluation response
            score, critique = parse_evaluation(eval)
            print(f"Score: {score}")
            print(f"Critique: {critique}")

            # log this to check output
            with open("output.txt", "a") as file:  # Use "w" to overwrite or "a" to append
                file.write(f"Critique: {critique}\nScore: {score}\nHLG: {highLevelGoals}\nLLG: {lowLevelGoals}\n\n\n")

            if score >= 8:
                print("Satisfactory score achieved! Breaking out of the loop.")
                break
        except ValueError as e:
                print(f"Error while parsing evaluation {i}: {e}")
    else:
            # If both evaluations fail to achieve a satisfactory score
            print("No satisfactory score achieved. Retrying...")
            feedback = Feedback(f"{actors}\n{highLevelGoals}\n{lowLevelGoals}", critique=critique)
            continue
        # Exit the outer loop if a satisfactory score was achieved
    if score >= 8:
        break

if attempts_count== MAX_ATTEMPTS and score < 8:
    print(f"Max attempts ({MAX_ATTEMPTS}) reached. Could not achieve a satisfactory score.")

print("API List STARTING...")
apiList = get_api_list_from_swagger()
print("API List DONE...")
print(apiList)

print("Mapping STARTING...")
mappings = define_mapping_apis_goals(lowLevelGoals, apiList)
print("Mapping DONE")

print("\n\n")
print_api_goal_mapping(mappings)

Description STARTING...
Description DONE...
description='The Genome Nexus API is a versatile tool designed to provide variant annotation through the use of HTTP requests. It supports clients in various programming languages, including Python, R, JavaScript, and TypeScript, along with a command line interface, all aimed at annotating MAF and VCF files. Comprehensive documentation can be accessed at https://docs.genomenexus.org/api.\n\nWeb-based tools are also available for the annotation of genetic variants, which are described in detail at https://docs.genomenexus.org/tools. The API offers long-term support primarily for the `/annotation` endpoint, which may be subject to changes in other parts over time.\n\nThe API provides capabilities such as:\n- Retrieving VEP annotation for lists of variants or dbSNPs.\n- Fetching canonical Ensembl Gene IDs based on Entrez Gene IDs or Hugo Symbols.\n- Looking up Ensembl Transcripts by IDs or protein or gene identifiers.\n- Accessing PDB header inf

KeyboardInterrupt: 